## Project 3

Dataset: COMPAS

In [159]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix

In [138]:
df = pd.read_csv("compas-scores-two-years.csv")
print(df.shape)
print(df.columns)

(7214, 53)
Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')


We will first fit a model predicting if the person will reoffend in the next two years. We could use a classifier for this. 

Next steps:

* look at the data for bias
* build a classifier

Sensitive variables: race, sex 

Target variable: two_year_recid

Features: age/age_cat, juv_fel_count, juv_misd_count, juv_other_count, priors_count, c_charge_degree, score_text, days_b_screening_arrest, decile_score, length_of_stay

## Twin test 

Feature: prior convictions

In [139]:
df = df[["race", "sex", "age_cat", "juv_fel_count", "juv_misd_count", "juv_other_count", "priors_count", "c_charge_degree", "two_year_recid"]]

df = df[(df["c_charge_degree"] != 'O')]

# sum 1-20 together
df["juv_fel_count"] = np.where(df["juv_fel_count"] == 0, 0, 1)
df["juv_misd_count"] = np.where(df["juv_misd_count"] == 0, 0, 1)
df["juv_other_count"] = np.where(df["juv_other_count"] == 0, 0, 1)
df["priors_count"]

df.loc[(df['priors_count'] >= 0) & (df['priors_count'] <= 10), 'priors_count'] = 0
df.loc[(df['priors_count'] > 10) & (df['priors_count'] <= 20), 'priors_count'] = 1
df.loc[(df['priors_count'] > 20), 'priors_count'] = 2

df["c_charge_degree"] = np.where(df["c_charge_degree"] == "M", 0, 1)
df["age_cat"] = df["age_cat"].replace({"Less than 25": 0, "25 - 45": 1, "Greater than 45": 2})
df["race"] = df["race"].replace({"Asian": "Minority", "Native American": "Minority", "Other": "Minority"})

df["sensitive"] = df.apply(lambda x: x["race"][0] + x["sex"][0], axis=1)

df.head()
df.to_csv("data_clean.csv")

C:\Users\annku\AppData\Local\Temp\ipykernel_10092\3782434972.py:16: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df["age_cat"] = df["age_cat"].replace({"Less than 25": 0, "25 - 45": 1, "Greater than 45": 2})


In [157]:
df = pd.read_csv("data_clean.csv")

df_count = df.groupby(["sensitive", "priors_count", "two_year_recid"]).agg(
    count=('race', 'count')).reset_index()

df_sum = df_count.groupby(["sensitive", "priors_count"]).agg(
    sum=('count', 'sum')).reset_index()

df_summary = pd.merge(df_count, df_sum, on=["sensitive", "priors_count"])

df_summary["prob"] = df_summary["count"] / df_summary["sum"]
print(df_summary)

   sensitive  priors_count  two_year_recid  count   sum      prob
0         AF             0               0    396   616  0.642857
1         AF             0               1    220   616  0.357143
2         AF             1               0      9    29  0.310345
3         AF             1               1     20    29  0.689655
4         AF             2               1      7     7  1.000000
5         AM             0               0   1278  2605  0.490595
6         AM             0               1   1327  2605  0.509405
7         AM             1               0     95   350  0.271429
8         AM             1               1    255   350  0.728571
9         AM             2               0     17    89  0.191011
10        AM             2               1     72    89  0.808989
11        CF             0               0    363   551  0.658802
12        CF             0               1    188   551  0.341198
13        CF             1               0      5    12  0.416667
14        

## Group fairness

only sensitive data as prior

In [158]:
df = pd.read_csv("data_clean.csv")

df_count = df.groupby(["sensitive", "two_year_recid"]).agg(
    count=('race', 'count')).reset_index()

df_sum = df_count.groupby(["sensitive"]).agg(
    sum=('count', 'sum')).reset_index()

df_summary = pd.merge(df_count, df_sum, on=["sensitive"])

df_summary["prob"] = df_summary["count"] / df_summary["sum"]
print(df_summary)

   sensitive  two_year_recid  count   sum      prob
0         AF               0    405   652  0.621166
1         AF               1    247   652  0.378834
2         AM               0   1390  3044  0.456636
3         AM               1   1654  3044  0.543364
4         CF               0    368   567  0.649030
5         CF               1    199   567  0.350970
6         CM               0   1120  1887  0.593535
7         CM               1    767  1887  0.406465
8         HF               0     70   103  0.679612
9         HF               1     33   103  0.320388
10        HM               0    335   534  0.627341
11        HM               1    199   534  0.372659
12        MF               0     54    73  0.739726
13        MF               1     19    73  0.260274
14        MM               0    221   354  0.624294
15        MM               1    133   354  0.375706


## Model

In [165]:
df = pd.read_csv("data_clean.csv")

# Select relevant columns for features and target
features = df.iloc[:, 3:-2]

target = df['two_year_recid']

# Handle missing values if any
features = features.fillna(0)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(report)

              precision    recall  f1-score   support

           0       0.64      0.81      0.72      1207
           1       0.64      0.44      0.52       958

    accuracy                           0.64      2165
   macro avg       0.64      0.62      0.62      2165
weighted avg       0.64      0.64      0.63      2165

